In [2]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [3]:
device = (
    "mps"
    if getattr(torch, "has_mps", False)
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


/tmp/ipykernel_5076/1229531653.py:3: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  if getattr(torch, "has_mps", False)


In [4]:
names = ['year', 'month', 'day', 'dec_year', 'sn_value', 'sn_error', 'obs_num', 'unuesed1']

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/SN_d_tot_V2.0.csv",
    sep=';', header=None, names=names,
    na_values=['-1'], index_col=False)
# starts with first none zero value
l = df[df['obs_num'] == 0].index.tolist()
start_id = max(l) + 40000
sorted_data = df[start_id:].copy()

In [5]:


sorted_data['sn_value'] = sorted_data['sn_value'].astype(float)
df_train = sorted_data[sorted_data['year'] < 2000]
df_test = sorted_data[sorted_data['year'] >= 2000]

spots_train = df_train['sn_value'].to_numpy().reshape(-1, 1)
spots_test = df_test['sn_value'].to_numpy().reshape(-1, 1)
print(spots_train.shape)
scaler = StandardScaler()
spots_train = scaler.fit_transform(spots_train).flatten().tolist()
spots_test = scaler.fit_transform(spots_test).flatten().tolist()
print(len(spots_test))



(15161, 1)
6391


In [6]:
def to_sequence(seq_size, obs):
  x = []
  y = []
  for i in range(len(obs) - seq_size):
    window = obs[i:(i + seq_size)]
    after_window = obs[i + seq_size]
    x.append(window)
    y.append(after_window)

  return torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), torch.tensor(y, dtype=torch.float32).view(-1,1,1) 

In [7]:
SEQUENCE_SIZE = 10
x_train, y_train = to_sequence(seq_size=SEQUENCE_SIZE, obs=spots_train)
x_test, y_test = to_sequence(seq_size=SEQUENCE_SIZE, obs=spots_test)
print(x_train.shape)


torch.Size([15151, 10, 1])


In [8]:
# Setup data loaders for batch
# Create a dataset that contains tensors
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)



In [8]:
1724*32

55168

In [9]:
# Positional encoding
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, dropout=0.1, max_len=5000):
    super(PositionalEncoding, self).__init__()
    self.dropout = nn.Dropout(p=dropout)

    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position*div_term)
    pe[:, 1::2] = torch.cos(position*div_term)
    # add dimensions and transpose TODO understand the dimensions
    pe = pe.unsqueeze(0).transpose(0,1)
    # This I don't understand
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x + self.pe[:x.size(0), :]
    return self.dropout(x)  


In [10]:
# Define model
class TransformerModel(nn.Module):
  def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
    super(TransformerModel, self).__init__()

    self.encoder = nn.Linear(input_dim, d_model)
    self.pos_encoder = PositionalEncoding(d_model=d_model, dropout=dropout, max_len=5000)
    encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
    self.transformer = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=num_layers)
    self.decoder = nn.Linear(d_model, 1)

  def forward(self, x):
    x = self.encoder(x)
    x = self.pos_encoder(x)
    x = self.transformer(x)
    x = self.decoder(x[:, -1, :])
    return x  
    

In [11]:
model = TransformerModel()

/home/hansalin/anaconda3/envs/myenv/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [13]:
# Train model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# TODO try to understand ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 2
early_stop_count = 0
min_val_loss = float('inf')  # TODO ????
val_losses = []
train_losses = []
for epoch in range(epochs):
  # set the model in training mode
  model.train()
  train_loss = []
  for batch in train_loader:
    x_batch, y_batch = batch
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    optimizer.zero_grad()
    outputs = model(x_batch)
    loss = criterion(outputs, y_batch)
    train_loss.append(loss.item()) 
    loss.backward()
    optimizer.step()

  train_loss = np.mean(train_loss)
  # validation
  # set model in evaluation mode
  val_loss = []
  
  # This part should not be part of the model thats the reason for .no_grad()
  with torch.no_grad():
    for batch in test_loader:
      x_batch, y_batch = batch
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)
      outputs = model(x_batch)
      loss = criterion(outputs, y_batch)
      val_loss.append(loss.item())   
          
    val_loss = np.mean(val_loss)
    scheduler.step(val_loss)

  if val_loss < min_val_loss:
      min_val_loss = val_loss
      early_stop_count = 0
  else:
      early_stop_count += 1

  if early_stop_count >= 5:
      print("Early stopping!")
      break
  
  print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss:.4f}")
  train_losses.append(train_loss)
  val_losses.append(val_loss)


/home/hansalin/anaconda3/envs/myenv/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/hansalin/anaconda3/envs/myenv/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([15, 1, 1])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/hansalin/anaconda3/envs/myenv/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([13, 1, 1])) that is different to the input size (torch.Size([13, 1])). This will likely lead to incorrect resu

Epoch 1/2, Training Loss: 0.9763040377243661, Validation Loss: 0.9782
Epoch 2/2, Training Loss: 0.9887852688881918, Validation Loss: 0.9893


In [14]:
# Evaluation
model.eval()
predictions = []
with torch.no_grad():
  for batch in test_loader:
    x_batch, y_batch = batch
    x_batch = x_batch.to(device)
    outputs = model(x_batch)
    predictions.extend(outputs.squeeze().tolist())
print() 

In [26]:
pred = np.array(predictions).reshape(-1,1)
print(pred.shape)
y = np.array(y_test.reshape(-1,1))
print(y.shape)

(6381, 1)
(6381, 1)


In [33]:
#print(predictions.shape)
print(np.array(predictions).reshape(-1, 1).shape)
print(scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).shape)

print(y_test.numpy().reshape(-1, 1).shape)
print(scaler.inverse_transform(y_test.numpy().reshape(-1, 1)).shape)


rmse = np.sqrt(np.mean((scaler.inverse_transform(np.array(predictions).reshape(-1, 1)) - scaler.inverse_transform(y_test.numpy().reshape(-1, 1)))**2))
print(f"Score (RMSE): {rmse:.4f}")

(6381, 1)
(6381, 1)
(6381, 1)
(6381, 1)
Score (RMSE): 64.7249


In [28]:
dif = (pred-y)
print(dif.shape)
square = dif**2
print(square.shape)
mse = np.sqrt(np.mean())
print(rmse)

(6381, 1)
(6381, 1)


TypeError: mean() missing 1 required positional argument: 'a'

In [16]:
y_test.reshape(-1,1)

tensor([[ 0.9242],
        [ 1.9314],
        [ 2.3653],
        ...,
        [-0.8732],
        [-0.9507],
        [-0.9662]])